# Milestone 2 Phase 2

- Nama : Naufal Aulia Adam
- Batch : 07
- email : naufal.auliadam@gmail.com

## Dataset Description

This is information about the lack of nutrients. Deficiency by nitrogen, phosphorus, potassium. By means of the lack of nutrients, there are methods as follows: To make the rice plants lack nitrogen Will add nutrients Phosphorus and potassium, but nitrogen is not added.

Kaggle: https://www.kaggle.com/guy007/nutrientdeficiencysymptomsinrice

GDrive: https://drive.google.com/drive/folders/1tI36IuOFkTpCx_tjlj-KGbusw8Gjet3y?usp=sharing

## Import Library

In [ ]:
!pip install split-folders

In [ ]:
import os
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import shutil
import random
import splitfolders
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D, Activation, GlobalMaxPool2D, BatchNormalization
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras import Input, Model
from tensorflow.keras.utils import plot_model
from sklearn.metrics import classification_report

## Data Loading

Data diunduh langsung dari Kaggle dengan menggunakan Kaggle Notebook.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Data yang sudah diunduh dapat dibagi menjadi data training, data validation, dan data test.

In [ ]:
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio("/kaggle/input/nutrientdeficiencysymptomsinrice/rice_plant_lacks_nutrients/", output="output",
    seed=1337, ratio=(.75, .2, .05), group_prefix=None, move=False) # default values

## Exploratory Data Analysis

In [ ]:
# Pengarahan lokasi folder
data_dir = pathlib.Path('/kaggle/working/output/train')
train_dir = '/kaggle/working/output/train'
val_dir = '/kaggle/working/output/val'
test_dir = '/kaggle/working/output/test'

In [ ]:
# Menampilkan label data
class_name = np.array(sorted([item.name for item in data_dir.glob("*")]))
print('Terdapat kelas: ', class_name)

In [ ]:
# Menampilkan jumlah data dalam folder
for dirpath, dirnames, filenames in os.walk('output'):
  print(f"Ada {len(dirnames)} folder dan ada {len(filenames)} gambar di dalam {dirpath}")

Gambar paling banyak berada dalam folder train dengan masing-masing berjumlah lebih besar dari 256, namun kurang dari 512. Jumlah ini dapat dijadikan acuan untuk menentukan node dari masing-masing layer yang akan dipakai.

In [ ]:
# Definisi fungsi untuk menampilkan gambar
def view_random_image(target_dir, target_class):
  target_folder = target_dir + target_class
  random_image = random.sample(os.listdir(target_folder), 1)

  img = mpimg.imread(target_folder + "/" + random_image[0])
  plt.imshow(img)
  plt.title(target_class)
  plt.axis("off")

  print(f"Image Shape : {img.shape}")
  return img

In [ ]:
# Menampilkan gambar secara random
img = view_random_image(target_dir='output/train/',
                        target_class=random.choice(class_name))

## Feature Engineering

In [ ]:
# Mengatur agar data yang teracak tetap sama
tf.random.set_seed(46)

# preprocess data
train_datagen = ImageDataGenerator(rescale=1/255.0)
valid_datagen = ImageDataGenerator(rescale=1/255.0)
test_datagen = ImageDataGenerator(rescale=1/255.0)

## Pipeline

In [ ]:
# flow from directory using datagen (Pipeline)
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=16,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               seed=46)

valid_data = valid_datagen.flow_from_directory(val_dir,
                                               batch_size=16,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               seed=46)
test_data = test_datagen.flow_from_directory(test_dir,
                                               batch_size=16,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               seed=46)

## Modelling

Pada pekerjaan ini akan digunakan aplikasi DenseNet121 sebagai model dasar (*base model*).

In [ ]:
# Definisi model
base_model = tf.keras.applications.DenseNet121(input_shape=(224, 224, 3),
                                                        include_top=False,
                                                        weights='imagenet')
base_model.trainable = False

In [ ]:
# Persiapan model
global_average_layer = GlobalAveragePooling2D()
layer = Dense(64, kernel_initializer=keras.initializers.LecunNormal(seed=46), activation='selu')
prediction_layer = Dense(3)
softmax = Activation('softmax')

inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = Dropout(0.25)(x)
x = global_average_layer(x)
#x = Dropout(0.25)(x)
#x = layer(x)
outputs = prediction_layer(x)
outputs = softmax(outputs)

model = Model(inputs, outputs)

In [ ]:
# Summary dari model
model.summary()
# Plot summary dari model
plot_model(model, show_shapes=True)

In [ ]:
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# train model
history = model.fit(train_data,
                    epochs=30,
                    validation_data=valid_data)

## Model Evaluation

Dalam mengevaluasi model, perlu dilakukan adanya penunjukkan grafik antara data training dengan data validation.

In [ ]:
# Definisi fungsi plot
def plot_loss_curves(history):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(loss))

  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title("Loss Curves")
  plt.xlabel("Epochs")
  plt.legend()

  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title("Accuracy Curves")
  plt.xlabel("Epochs")
  plt.legend()

In [ ]:
# Menampilkan hasil plot
print("Base Model")
plot_loss_curves(history)

Terlihat bahwa terdapat sedikit celah antara grafik data training dengan grafik data validation. Hal ini mengindikasikan bahwa terjadi sedikit *overfit* pada model, sehingga perlu dilakukan Model Improvement.

## Model Improvement

Model Improvement dapat dilakukan dengan mengaugementasi gambar pada data training.

In [ ]:
# train dengan augmentasi
train_datagen_augmented = ImageDataGenerator(rescale=1/255.0,
                                             rotation_range=0.2,
                                             zoom_range=0.2,
                                             width_shift_range=0.2,
                                             height_shift_range=0.2,
                                             horizontal_flip=True)

# flow from directory using datagen (Pipeline)
train_data_augmented = train_datagen_augmented.flow_from_directory(train_dir,
                                               batch_size=16,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               seed=46)

In [ ]:
# Definisi model
base_model = tf.keras.applications.DenseNet121(input_shape=(224, 224, 3),
                                                        include_top=False,
                                                        weights='imagenet')
base_model.trainable = False

In [ ]:
# Persiapan model
global_average_layer = GlobalAveragePooling2D()
layer = Dense(64, kernel_initializer=keras.initializers.LecunNormal(seed=46), activation='selu')
prediction_layer = Dense(3)
softmax = Activation('softmax')

inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = Dropout(0.25)(x)
x = global_average_layer(x)
#x = Dropout(0.25)(x)
#x = layer(x)
outputs = prediction_layer(x)
outputs = softmax(outputs)

model_improv = Model(inputs, outputs)

In [ ]:
# Summary dari model
model_improv.summary()
# Plot summary dari model
plot_model(model_improv, show_shapes=True)

Selain dengan augmentasi gambar, dapat juga dilakukan pengaturan pada learning rate-nya. Tentunya hal ini akan memakan waktu yang lebih lama dari sebelumnya, sehingga jumlah epoch-nya dapat diperbanyak.

In [ ]:
# Definisi optimizer
optimizer = Adam(learning_rate = 0.00001)

# compile model
model_improv.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

# train model
history_improv = model_improv.fit(train_data_augmented,
                    epochs=175,
                    validation_data=valid_data)

In [ ]:
# Menampilkan hasil plot
print("New Model")
plot_loss_curves(history_improv)

Terlihat bahwa model yang dihasilkan jauh lebih baik dari sebelumnya.

## Model Inference

In [ ]:
tes_data = test_data.next()

In [ ]:
res = model.predict(tes_data[0])
y_test = np.array(tes_data[1]).argmax(axis=1)
res = res.argmax(axis=1)
print(classification_report(y_test, res))

## Conclusion

Dataset yang digunakan adalah dataset untuk memprediksi nutrisi yang kurang dari sebuah tanaman padi. Cukup sulit dalam membangun model prediksinya, karena secara kasat mata, tidak terlalu ada perbedaan yang signifikan antara satu sampel dengan yang lain. Hal ini menjadi pertimbangan yang sulit apakah ingin membangun model berdasarkan model yang *deep* atau tidak. Model DenseNet121 merupakan model yang cukup *deep*, namun memiliki ukuran dan parameter yang tidak terlalu banyak, oleh karena itu model tersebut dipakai dalam pekerjaan ini.

Dataset ini cukup besar, namun label-label yang ada masih tergolong kurang, terutama mengenai dataset terkait padi yang cukup akan nutrisi. Harapannya, dataset ini dapat berkembang sehingga dalam proyek mendatang dapat dibangun model yang lebih membantu orang-orang dalam mengklasifikasi tanaman padi.